In [1]:
import sys
sys.path.append("..")

This notebook will give an example how to use Renormalizer to calculate the ground state energy of the open boundary spin 1/2 1-D Heisenberg model.

$H = J \sum_i [S_i^z S_{i+1}^z + \frac{1}{2}(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)]$

exact result (Bethe Anstatz):

$L$       $E/J$

16	-6.9117371455749

24	-10.4537857604096

32	-13.9973156182243

48	-21.0859563143863

64	-28.1754248597421


In [2]:
from renormalizer import Model, Op, BasisHalfSpin,  Mps, Mpo, optimize_mps

# define the # of spins
nspin = 32

# define the model
# sigma^+ = S^+
# sigma^- = S^-
# 1/2 sigma^x,y,z = S^x,y,z

ham_terms = []
for ispin in range(nspin-1):
    op1 = Op("sigma_z sigma_z", [ispin, ispin+1], 1.0/4)
    op2 = Op("sigma_+ sigma_-", [ispin, ispin+1], 1.0/2)
    op3 = Op("sigma_- sigma_+", [ispin, ispin+1], 1.0/2)
    ham_terms.extend([op1, op2, op3])

# set the spin order and local basis
basis = [BasisHalfSpin(i) for i in range(nspin)]

# construct Hamiltonian MPO
model = Model(basis, ham_terms)
mpo = Mpo(model)
print(f"mpo_bond_dims:{mpo.bond_dims}")

# set the sweep paramter
M = 30
procedure = [[M, 0.2], [M, 0], [M, 0], [M,0], [M,0]]

# initialize a random MPS
qntot = 0
mps = Mps.random(model, qntot, M)

mps.optimize_config.procedure = procedure
mps.optimize_config.method = "2site"

# optimize MPS
energies, _ = optimize_mps(mps.copy(), mpo)
print("gs energy:", min(energies))


2022-10-05 17:43:32,267[INFO] Use NumPy as backend
2022-10-05 17:43:32,268[INFO] use 64 bits
2022-10-05 17:43:32,301[DEBUG] # of operator terms: 93
2022-10-05 17:43:32,302[DEBUG] symbolic mpo algorithm: Hopcroft-Karp
2022-10-05 17:43:32,303[DEBUG] Input operator terms: 93
2022-10-05 17:43:32,319[DEBUG] After combination of the same terms: 93
2022-10-05 17:43:32,463[INFO] optimization method: 2site
2022-10-05 17:43:32,464[INFO] e_rtol: 1e-06
2022-10-05 17:43:32,465[INFO] e_atol: 1e-08
2022-10-05 17:43:32,466[INFO] procedure: [[30, 0.2], [30, 0], [30, 0], [30, 0], [30, 0]]
2022-10-05 17:43:32,502[DEBUG] isweep: 0
2022-10-05 17:43:32,503[DEBUG] mmax, percent: 30, 0.2
2022-10-05 17:43:32,504[DEBUG] mps current size: 329.7KiB, Matrix product bond dim:[1, 2, 4, 8, 16, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 16, 8, 4, 2, 1]
2022-10-05 17:43:32,505[DEBUG] optimize site: [0, 1]
2022-10-05 17:43:32,506[DEBUG] use direct eigensolver
2022-10-05 1

mpo_bond_dims:[1, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 1]


2022-10-05 17:43:32,851[DEBUG] energy: -2.8035001959542845
2022-10-05 17:43:32,855[DEBUG] optimize site: [4, 5]
2022-10-05 17:43:32,946[DEBUG] use davidson, HC hops: 54
2022-10-05 17:43:32,949[DEBUG] energy: -3.2163841920427214
2022-10-05 17:43:32,953[DEBUG] optimize site: [5, 6]
2022-10-05 17:43:33,096[DEBUG] use davidson, HC hops: 69
2022-10-05 17:43:33,097[DEBUG] energy: -3.644635697295872
2022-10-05 17:43:33,102[DEBUG] optimize site: [6, 7]
2022-10-05 17:43:33,206[DEBUG] use davidson, HC hops: 49
2022-10-05 17:43:33,207[DEBUG] energy: -4.041841441126216
2022-10-05 17:43:33,212[DEBUG] optimize site: [7, 8]
2022-10-05 17:43:33,317[DEBUG] use davidson, HC hops: 50
2022-10-05 17:43:33,319[DEBUG] energy: -4.488923472286725
2022-10-05 17:43:33,324[DEBUG] optimize site: [8, 9]
2022-10-05 17:43:33,426[DEBUG] use davidson, HC hops: 49
2022-10-05 17:43:33,428[DEBUG] energy: -4.956935755277623
2022-10-05 17:43:33,433[DEBUG] optimize site: [9, 10]
2022-10-05 17:43:33,514[DEBUG] use davidson, H

gs energy: -13.997315090070297
